In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
config_path = '/home/hyz/da/testhub/cmr_test/'

#### 检查flg是否包含16的函数

In [3]:
def flg2num(flg):
    flg_list = []
    for i in np.linspace(8,0,9):
        num_i = 2**i
        if flg >= num_i:
            flg = flg%num_i
            flg_list.append(num_i)
    return flg_list

def flgcheck16(flg):
    flg_list = flg2num(flg)
    flg_list = np.array(flg_list)
    if len(flg_list[flg_list==16.]) == 1:
        return 1
    else:
        return 0

def flgncheck16(flg):
    for i in np.linspace(8,5,4,dtype=int):
        num_i = 2**i
        flg %= num_i
    flg //= 16
    return flg

flgncheck16(np.array([[257,234,412,25,156],[1,0,24,36,76]]))

array([[0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0]])

#### 像素数计算函数

In [4]:
def mulapix():
    with open(config_path+"test_file_list.txt") as tfl:
        flist = tfl.readlines()
    epoch_idl, ccd_idl , nump_flgl, nump_crsl, nump_checkl = [],[],[],[],[]
    
    for _file in flist:
        _file = _file.rstrip()
        _crsf = _file.split('SCI')[0]+"CRS"+_file.split('SCI')[1]+"L0_1.fits"
        _flgf = _file + 'flg.fits.sisteen'      # flg提取标记16后图
        # _flgf = _file + 'flg.fits'             # flg原图
        _checkf = _file + 'check.fits'
        with fits.open(_crsf) as hdu_crs:
            crs = hdu_crs[1].data
            if os.path.exists(_flgf) is False:
                print("cp %s %s" %(_flgf[:-8], _flgf))
                os.system("cp %s %s" %(_flgf[:-8], _flgf))
                with fits.open(_flgf,mode='update') as hdu_flg:
                    flg = hdu_flg[1].data
                    flg = flgncheck16(flg)
                    flg = np.int8(flg)
                    hdu_flg[1].data = flg
                    hdu_flg.flush()
            else:
                with fits.open(_flgf) as hdu_flg:
                    flg = hdu_flg[1].data
            check = crs*flg
            hdu_crs[1].data = check
            hdu_crs.writeto(_checkf, overwrite=True)
        nump_flg = np.sum(flg)
        nump_check = np.sum(np.ones_like(check)[check>0])
        nump_crs = np.sum(np.ones_like(crs)[crs>0])
        epoch_idl.append(_file[-6:-4])
        ccd_idl.append(_file[-3:-1])
        nump_flgl.append(nump_flg)
        nump_crsl.append(nump_crs)
        nump_checkl.append(nump_check)
    pix_table = pd.DataFrame({  'epoch_id':epoch_idl,
                                'ccd_id':ccd_idl,
                                'pixel_crs':nump_crsl,
                                'pixel_flg':nump_flgl,
                                'pixel_check':nump_checkl  })
    pix_table["pixel-loss find rate"] = (1-pix_table['pixel_check']/pix_table['pixel_crs'])*100
    pix_table['pixel-false alarm rate'] = (pix_table['pixel_flg']-pix_table['pixel_check'])/pix_table['pixel_flg']*100
    return pix_table

    

#### sex运行与reg生成函数

In [5]:
def sexaread():

    with open(config_path+"test_file_list.txt") as tfl:
        flist = tfl.readlines()

    epoch_idl, ccd_idl, sex_crsl, sex_checkl, sex_flgl = [], [], [], [], []
    
    sufl = ['crs','check','flg']
    for _file in flist:
        _file = _file.rstrip()
        epoch_idl.append(_file[-6:-4])
        ccd_idl.append(_file[-3:-1])
        for _suf in sufl:
            _acat = config_path + 'acat/' + _file[-6:] + _suf + '.acat'
            if _suf == 'crs':
                file = _file.split('SCI')[0]+"CRS"+_file.split('SCI')[1]+"L0_1.fits"
            elif _suf == 'check':
                file = _file + _suf+".fits"
            elif _suf == 'flg':
                file = _file + _suf + '.fits.sisteen'      # flg提取图
                # file = _file.rstrip() + _suf + '.fits'              # flg原图
            print(file,"\n",_acat)
            
            os.system("source-extractor -c " + config_path + "cmr.sex " + file    
                    + ' -CATALOG_NAME ' + _acat)
            
            with fits.open(_acat) as hdus:
                cat = Table(hdus[2].data)
                lenc = len(cat)
                _reg = _acat[0:-5] + '.reg'
                with open(_reg, 'w') as f:
                    f.write("# Region file format: DS9 version 4.1\nglobal color=green dashlist=8 3 width=1 font='helvetica 10 normal roman' select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nimage\n")
                    for i in range(lenc):
                        f.writelines("ellipse("+str(cat['XWIN_IMAGE'][i])+","+str(cat['YWIN_IMAGE'][i])+
                                    ","+str(4*cat['AWIN_IMAGE'][i])+","+str(4*cat['BWIN_IMAGE'][i])+","
                                    +str(cat['THETAWIN_IMAGE'][i])+")\n")
            if _suf == 'crs':
                sex_crsl.append(lenc)
            if _suf == 'check':
                sex_checkl.append(lenc)
            elif _suf == 'flg':
                sex_flgl.append(lenc)
    sex_table = pd.DataFrame({  'epoch_id': epoch_idl,
                                'ccd_id': ccd_idl,
                                'source_crs': sex_crsl,
                                'source_flg': sex_flgl,
                                'source_check': sex_checkl  })
    sex_table["source-loss find rate"] = (1-sex_table['source_check']/sex_table['source_crs'])*100
    sex_table['source-false alarm rate'] = (sex_table['source_flg']-sex_table['source_check'])/sex_table['source_flg']*100
    return sex_table


#### 检查reg差异并生成新的reg函数

In [6]:
def ergsub(former,latter,new):
    with open(former,'r') as erg1:
        with open(latter,'r') as erg2:
            with open(new,'w') as erg_n:
                reg_list = erg1.readlines()
                reg_listc = reg_list.copy()
                reg1_list = erg2.readlines()
                for line in reg_list:
                    r1 = line.split("(",1)
                    if len(r1) > 1:
                        r1 = r1[1]
                        r1 = r1.split(",",4)
                        r1x = float(r1[0])
                        r1y = float(r1[1])
                        for line1 in reg1_list:
                            r2 = line1.split("(",1)
                            if len(r2) > 1:
                                r2 = r2[1]
                                r2 = r2.split(",",4)
                                r2x = float(r2[0])
                                r2y = float(r2[1])
                                if abs(r1x-r2x)+abs(r1y-r2y) < 1:
                                    reg_listc.remove(line)
                                    reg1_list.remove(line1)
                                    break
                                
                erg_n.writelines("# Region file format: DS9 version 4.1\nglobal color=yellow dashlist=8 3 width=1 font='helvetica 10 normal roman' select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nimage\n")
                erg_n.writelines(reg1_list[3:])
                erg_n.writelines("# Region file format: DS9 version 4.1\nglobal color=magenta dashlist=8 3 width=1 font='helvetica 10 normal roman' select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nimage\n")
                erg_n.writelines(reg_listc[3:])
                erg_n.writelines("text 4500 8900 {yellow: later-former} # color=white \ntext 4500 8800 {magenta: former-later} # color=white")
                num_con1 = len(reg_listc)-3
                num_con2 = len(reg1_list)-3
                return [num_con1,num_con2]

def regdiff():
    with open(config_path+"test_file_list.txt") as tfl:
        flist = tfl.readlines()
    num_ccl,num_fcl,num_cfl = [],[],[]
    for _file in flist:
        _pre = _file.split("CSST",1)[0] + "/acat/" + _file[-6:]
        _crsr = _pre+"crs.reg"
        _flgr = _pre+"flg.reg"
        _checkr = _pre+"check.reg"
        _cc = _pre+"crs-check.reg"
        _fc = _pre+"flg-check.reg"
        _cf = _pre+"crs-flg.reg"
        num_ccl.append(ergsub(_crsr,_checkr,_cc))
        num_fcl.append(ergsub(_flgr,_checkr,_fc))
        num_cfl.append(ergsub(_crsr,_flgr,_cf))
    num_ccl = np.array(num_ccl)
    num_fcl = np.array(num_fcl)
    num_cfl = np.array(num_cfl)
    rd_table = pd.DataFrame({   'crs-check':num_ccl[:,0],
                                'check-crs':num_ccl[:,1],
                                'flg-check':num_fcl[:,0],
                                'check-flg':num_fcl[:,1],
                                'crs-flg':num_cfl[:,0],
                                'flg-crs':num_cfl[:,1]})
    return rd_table



#### 运行

In [7]:
pix_table = mulapix()
sex_table = sexaread()
a_table = pd.concat([pix_table,sex_table],axis=1)
a_table = a_table.T.drop_duplicates().T
idl = []
bandl = []
for i in range(20):
    if i != 4 and i != 15:
        id = i+6
        idl.append('25_%02d' % (i+6))
        if id==6 or id==25 or id==11 or id==20:
            bandl.append('y')
        elif id==7 or id==24:
            bandl.append('i')
        elif id==8 or id==23:
            bandl.append('g')
        elif id==9 or id==22:
            bandl.append('r')
        elif id==12 or id==13 or id==18 or id==19:
            bandl.append('NUV')
        elif id==14 or id==17:
            bandl.append('u')
        elif id==15 or id==16:
            bandl.append('z')
a_table['id'] = idl
a_table.insert(1,'band',bandl,allow_duplicates=True)
a_table

cp /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_13_flg.fits /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_13_flg.fits.sisteen
cp /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_14_flg.fits /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_14_flg.fits.sisteen
cp /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_15_flg.fits /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_15_flg.fits.sisteen
cp /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_16_flg.fits /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_16_flg.fits.sisteen
cp /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_17_flg.fits /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_2024121208


> WARNING: NTHREADS != 1 ignored: this build of Source Extractor is single-threaded

> 
----- Source Extractor 2.25.0 started on 2022-11-22 at 21:41:02 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_06_L0_1.fits
----- Measuring from: CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_06_L0_1.fits [1/1]
      "Unnamed" / no ext. header / 9216x9232 / 16 bits (integers)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  256
> Setting up background map at line:  512
> Setting up background map at line:  768
> Setting up background map at line: 1024
> Setting up background map at line: 1280
> Setting up background map at line: 1536
> Setting up background map at line: 1792
> Setting up background map at line: 2048
> Setting up background map at line: 2304
> Setting up background map at line: 2560
> Setting up background map a

/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_06_check.fits 
 /home/hyz/da/testhub/cmr_test/acat/25_06_check.acat



> WARNING: NTHREADS != 1 ignored: this build of Source Extractor is single-threaded

> 
----- Source Extractor 2.25.0 started on 2022-11-22 at 21:41:08 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_06_check.fits
----- Measuring from: CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_06_check.fits [1/1]
      "Unnamed" / no ext. header / 9216x9232 / 16 bits (integers)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  256
> Setting up background map at line:  512
> Setting up background map at line:  768
> Setting up background map at line: 1024
> Setting up background map at line: 1280
> Setting up background map at line: 1536
> Setting up background map at line: 1792
> Setting up background map at line: 2048
> Setting up background map at line: 2304
> Setting up background map at line: 2560
> Setting up background map

/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_06_flg.fits.sisteen 
 /home/hyz/da/testhub/cmr_test/acat/25_06_flg.acat


...


> WARNING: NTHREADS != 1 ignored: this build of Source Extractor is single-threaded

> 
----- Source Extractor 2.25.0 started on 2022-11-22 at 21:46:42 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_25_L0_1.fits
----- Measuring from: CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_25_L0_1.fits [1/1]
      "Unnamed" / no ext. header / 9216x9232 / 16 bits (integers)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  256
> Setting up background map at line:  512
> Setting up background map at line:  768
> Setting up background map at line: 1024
> Setting up background map at line: 1280
> Setting up background map at line: 1536
> Setting up background map at line: 1792
> Setting up background map at line: 2048
> Setting up background map at line: 2304
> Setting up background map at line: 2560
> Setting up background map a

/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_25_check.fits 
 /home/hyz/da/testhub/cmr_test/acat/25_25_check.acat



> WARNING: NTHREADS != 1 ignored: this build of Source Extractor is single-threaded

> 
----- Source Extractor 2.25.0 started on 2022-11-22 at 21:46:47 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_25_check.fits
----- Measuring from: CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_25_check.fits [1/1]
      "Unnamed" / no ext. header / 9216x9232 / 16 bits (integers)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  256
> Setting up background map at line:  512
> Setting up background map at line:  768
> Setting up background map at line: 1024
> Setting up background map at line: 1280
> Setting up background map at line: 1536
> Setting up background map at line: 1792
> Setting up background map at line: 2048
> Setting up background map at line: 2304
> Setting up background map at line: 2560
> Setting up background map

/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_25_flg.fits.sisteen 
 /home/hyz/da/testhub/cmr_test/acat/25_25_flg.acat



> WARNING: NTHREADS != 1 ignored: this build of Source Extractor is single-threaded

> 
----- Source Extractor 2.25.0 started on 2022-11-22 at 21:46:53 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_25_flg.fits.sisteen
----- Measuring from: CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_25_flg.fits.sisteen [1/1]
      "Unnamed" / no ext. header / 9216x9232 / 16 bits (integers)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  256
> Setting up background map at line:  512
> Setting up background map at line:  768
> Setting up background map at line: 1024
> Setting up background map at line: 1280
> Setting up background map at line: 1536
> Setting up background map at line: 1792
> Setting up background map at line: 2048
> Setting up background map at line: 2304
> Setting up background map at line: 2560
> Setting up ba

In [16]:
a_table.sort_values('source-loss find rate')

,id,pixel_crs,pixel_flg,pixel_check,pixel-loss find rate,pixel-false alarm rate,source_crs,source_flg,source_check,source-loss find rate,source-false alarm rate,band
2,25_08,270812,248428,240902,11.044562,3.029449,11345,11250,11305,0.352578,-0.488889,g
15,25_23,270383,249163,241880,10.541713,2.922986,11382,11258,11330,0.456862,-0.639545,g
16,25_24,271545,243937,236986,12.726804,2.849506,11252,11105,11121,1.164237,-0.144079,i
14,25_22,270121,287197,235919,12.661733,17.854643,11253,11064,11111,1.261886,-0.424801,r
3,25_09,271314,347862,236637,12.781132,31.973886,11355,11079,11210,1.27697,-1.182417,r
4,25_11,270366,240054,233783,13.530917,2.612329,11282,11097,11128,1.365006,-0.279355,y
13,25_20,270537,300079,234190,13.435131,21.957218,11293,11094,11134,1.407952,-0.360555,y
1,25_07,270431,243123,235831,12.794391,2.999305,11212,11014,11051,1.435961,-0.335936,i
9,25_16,269836,229817,224188,16.916942,2.44934,11251,11031,11044,1.839836,-0.11785,z
8,25_15,270688,296219,225420,16.723312,23.900898,11339,11052,11124,1.896111,-0.651466,z


In [ ]:
rd_table = regdiff()
c_table = pd.concat([a_table,rd_table],axis=1)
c_table['diff-loss find rate'] = (c_table['crs-check']-c_table['check-crs'])/c_table['source_crs']*100
c_table['diff-false alarm rate'] = (c_table['flg-check']-c_table['check-flg'])/c_table['source_check']*100
c_table


,id,pixel_crs,pixel_flg,pixel_check,pixel-loss find rate,pixel-false alarm rate,source_crs,source_flg,source_check,source-loss find rate,source-false alarm rate,crs-check,check-crs,flg-check,check-flg,crs-flg,flg-crs,diff-loss find rate,diff-false alarm rate
0,25_12,269980,223026,217668,19.37625,2.40241,11166,10814,10853,2.803152,-0.360644,446,133,375,414,779,427,2.803152,-0.359348


In [ ]:
c_table.T

,0
id,25_12
pixel_crs,269980
pixel_flg,223026
pixel_check,217668
pixel-loss find rate,19.37625
pixel-false alarm rate,2.40241
source_crs,11166
source_flg,10814
source_check,10853
source-loss find rate,2.803152


In [ ]:
stop

NameError: name 'stop' is not defined

#### 查看检测结果与测试

* flg-check
* cmr-check
* cmr-flg

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits.sisteen \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_flg-check.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_check.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_flg-check.reg")

0

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-check.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_check.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-check.reg")

0

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-flg.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits.sisteen \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-flg.reg")

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-flg.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-flg.reg")

读取acat并生成erg和记录表

In [ ]:
file = '/home/hyz/da/testhub/cmr_test///CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits'
file = file.split('SCI')[0]+"CRS"+file.split('SCI')[1].rstrip()[:-8]+"L0_1.fits"
os.system("ds9 %s"% file)

0

In [ ]:
'flg.fits.sisteen'[:-8]

'flg.fits'

* 测试

In [ ]:
with fits.open("CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits") as hdu_flg:
    flg = hdu_flg[1].data
np.sum(flg)

3585092

目录生成

In [17]:
for j in range(20):
    j += 20
    for i in range(20):
        if i != 4 and i != 15:
            print('/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_1000000%02d_%02d' %(j,i+6))

/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_06
/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_07
/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_08
/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_09
/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_11
/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_12
/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_13
/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_14
/media/hyz/Elements/testhub/cmr/data_testgroup/CSST_MSC_MS_SCI_20241204123427_20241204123657_100000020_15
/media/hyz/Elements/testhub/cmr/data_testgroup

In [ ]:
"CHECK" in "cmr_test/CSST_CRSCHECK_25_12.fits"

True